In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch 
from torch import nn
from torch import optim
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM,GemmaTokenizer
import pandas as pd
import gradio as gr
from transformers import pipeline
from peft import*

In [2]:
model = AutoModelForCausalLM.from_pretrained('gemma_2b_peft_tuned')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained('google/gemma-2b')

In [4]:
model.to('cuda')

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=256, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDi

In [5]:
def generate_text(input_txt):
    format_txt = f'Lyrics: {input_txt}'
    inputs = tokenizer(format_txt,return_tensors="pt").to('cuda')
    outputs = model.generate(**inputs,max_new_tokens=50,do_sample=True,temperature=0.8,top_k=50,top_p=0.95)
    generated_txt = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_txt

In [8]:
#try it out
sample_input = "I love eating peanut butter more than you do."
print(generate_text(sample_input),"\n")

Lyrics: I love eating peanut butter more than you do.
You're the peanut butter to my jelly, and I love you so.
I love you so.
I love you so, I'd eat peanut butter with you.
(Peanut butter is good with jelly.)
You and 



In [31]:
lyrics_interface = gr.Interface(fn=generate_text, inputs="text", outputs="text",
                                title="Gemma-2B-PEFT-Tuned with Taylor Swift Lyrics",
                                description="Enter a sentence to generate lyrics. Model used -Google/gemma-2b",
                                examples=[["We were in love"],["I'm feeling good"],["I'm feeling bad"]])
lyrics_interface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [32]:
lyrics_interface.close()

Closing server running on port: 7860
